In [49]:
import pandas as pd
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow  import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Softmax, Conv2D, MaxPooling2D,BatchNormalization,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt

In [50]:
import pandas as pd
data = pd.read_csv('sign_mnist_train.csv')
#data = pd.read_csv('sign_data2.csv')
data.shape

(27455, 785)

In [51]:
Y = data['label']
X = data.drop('label',1)

In [52]:
myDict = {0: 'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J', 10:'K', 11:'L', 12:'M',
          13: 'N', 14:'O', 15:'P', 16:'Q', 17:'R', 18:'S', 19:'T', 20:'U', 21:'V', 22:'W', 23:'X', 24:'Y', 25:'Z'}

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [56]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')
X_train = X_train/255
X_test = X_test/255



In [57]:
type(X_train)

pandas.core.frame.DataFrame

In [60]:
#CNN model
X_train = np.array(X_train).reshape(-1,28,28,1)
X_test = np.array(X_test).reshape(-1,28,28,1)

In [61]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

In [62]:
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 24 , activation = 'softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 75)        750       
_________________________________________________________________
batch_normalization_12 (Batc (None, 28, 28, 75)        300       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 75)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 14, 14, 50)        33800     
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 50)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 14, 14, 50)        200       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 7, 7, 50)         

In [63]:
# With data augmentation to prevent overfitting

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)



In [64]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

In [65]:



print(X_train.shape)
print(type(X_train))
print(X_test.shape)
print(type(X_test))

print(y_train.shape)
print(type(y_train))
print(y_test.shape)
print(type(y_test))

(18394, 28, 28, 1)
<class 'numpy.ndarray'>
(9061, 28, 28, 1)
<class 'numpy.ndarray'>
(18394, 24)
<class 'numpy.ndarray'>
(9061, 24)
<class 'numpy.ndarray'>


In [66]:
history = model.fit(datagen.flow(X_train,y_train, batch_size = 128) ,epochs = 3 , validation_data = (X_test, y_test) , callbacks = [learning_rate_reduction])

Epoch 1/3
 94/144 [==================>...........] - ETA: 18s - loss: 1.7905 - accuracy: 0.4559

KeyboardInterrupt: 